In [0]:
# local computer didn't have the manpower for elmo embedding
# so all of this was done on google colab

!pip install tensorflow==1.14
!pip install tensorflow_hub
!pip install -U wget
!pip install -U spacy
!python -m spacy download en_core_web_md
!pip install rouge_score

In [0]:
import re
import os
from zipfile import ZipFile
import wget
import glob

from rouge_score import rouge_scorer
from gensim.summarization.summarizer import summarize
from tqdm import tqdm
import spacy

import nltk
#nltk.download('wordnet')
#nltk.download('stopwords')
import numpy as np
from collections import defaultdict
import random

from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

import tensorflow_hub as hub
import tensorflow as tf
import en_core_web_sm

from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import silhouette_score, silhouette_samples
from sklearn.cluster import SpectralClustering
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import MeanShift


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd '/content/gdrive/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
/content/gdrive


In [0]:
apple_item1 = open('My Drive/AAPL_0000320193_20171230_item1.txt', encoding='utf-8').read().splitlines()
apple_item2 = open('My Drive/AAPL_0000320193_20171230_item2.txt', encoding='utf-8').read().splitlines()
apple_item3 = open('My Drive/AAPL_0000320193_20171230_item3.txt', encoding='utf-8').read().splitlines()
apple_item4 = open('My Drive/AAPL_0000320193_20171230_item4.txt', encoding='utf-8').read().splitlines()
apple_part2 = open('My Drive/AAPL_0000320193_20171230_part2.txt', encoding='utf-8').read().splitlines()

apple_part1 = apple_item1 + apple_item2 + apple_item3 + apple_item4

195


In [0]:
apple_item1_text = open('My Drive/AAPL_0000320193_20171230_item1.txt', encoding='utf-8').read()
apple_item2_text = open('My Drive/AAPL_0000320193_20171230_item2.txt', encoding='utf-8').read()

'S'

In [0]:
elmo = hub.Module("https://tfhub.dev/google/elmo/3", trainable=True)
embeddings = elmo(apple_part1, signature = "default", as_dict=True)['elmo']

In [0]:
print(embeddings.shape)

(195, 233, 1024)


In [0]:
def elmo_vectors(x):
  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    
    # return average of ELMo features
    return sess.run(tf.reduce_mean(embeddings,1))


In [0]:
elmo_train = elmo_vectors(embeddings)

In [0]:
sentences = apple_part1

In [0]:
## List <String> model(String doc_text, int num_sents)

"""
The order of the candidate sentences in the summary is determined by the positions of the
sentences in their corresponding clusters in the original email. For example,
a candidate sentence is chosen as the first line in the summary if most of the
sentences that lie in its cluster occur at the beginning of the email.

Sentences:
1. Apple inc. designs, manufactures and markets mobile communication...
2. The company's business is subject to risks of international operations...
3. During the three months ended December 30, 2017, cash generated by operating
activities of $28.3 billion was a result of $20.1 billion of net income
"""

def kmeans(doc_text, num_sents):
  kmeans = KMeans(num_sents).fit(elmo_train)
  # for each cluster, it finds the sentences associated with it,
  # and chooses the median one
  avg = []
  for j in range(num_sents):
      idx = np.where(kmeans.labels_ == j)[0]
      avg.append(np.median(idx))
  closest, _ = pairwise_distances_argmin_min(kmeans.cluster_centers_, elmo_train)
  ordering = sorted(range(num_sents), key=lambda k: avg[k])
  summary = [doc_text[closest[idx]] for idx in ordering]
  return summary

kmeans(sentences, 10)

['The Company may use derivatives to partially offset its business exposure to foreign currency and interest rate risk on expected future cash flows, on net investments in certain foreign subsidiaries and on certain existing assets and liabilities. However, the Company may choose not to hedge certain exposures for a variety of reasons including, but not limited to, accounting considerations and the prohibitive economic cost of hedging particular exposures. There can be no assurance the hedges will offset more than a portion of the financial impact resulting from movements in foreign currency exchange or interest rates.\n',
 'The Company considers the declines in market value of its marketable securities investment portfolio to be temporary in nature. The Company typically invests in highly rated securities, and its investment policy generally limits the amount of credit exposure to any one issuer. The policy generally requires investments to be investment grade, with the primary object

In [0]:
# default: ward linkage
def agg_clustering(doc_text, num_sents): 
  #average = []
  summaries = defaultdict(list)
  map = defaultdict(list)
  # linkages: ward, complete, single
  ac = AgglomerativeClustering(n_clusters=num_sents, linkage='ward').fit_predict(elmo_train)
  for i in range(len(ac)):
    map[ac[i]].append(doc_text[i])
  for key in map:
    summaries[num_sents].append(random.choice(map[key]))

  return summaries[num_sents]

agg_clustering(sentences, 10)

['Debt',
 'Apple Inc. and its wholly-owned subsidiaries (collectively “Apple” or the “Company”) designs, manufactures and markets mobile communication and media devices and personal computers, and sells a variety of related software, services, accessories, networking solutions and third-party digital content and applications. The Company’s products and services include iPhone®, iPad®, Mac®, Apple Watch®, Apple TV®, a portfolio of consumer and professional software applications, iOS, macOS®, watchOS® and tvOS™ operating systems, iCloud®, Apple Pay® and a variety of accessory, service and support offerings. The Company sells and delivers digital content and applications through the iTunes Store®, App Store®, Mac App Store, TV App Store, iBooks Store® and Apple Music® (collectively “Digital Content and Services”). The Company sells its products worldwide through its retail stores, online stores and direct sales force, as well as through third-party cellular network carriers, wholesalers, 

In [0]:
def spectral_clustering(doc_text, num_sents): 
  summaries = defaultdict(list)
  map = defaultdict(list)
  ac = SpectralClustering(n_clusters=num_sents).fit_predict(elmo_train)  
  for i in range(len(ac)):
     map[ac[i]].append(sentences[i])
  for key in map:
     summaries[num_sents].append(random.choice(map[key]))
  return summaries[num_sents]

spectral_clustering(sentences, 10)

['See accompanying Notes to Condensed Consolidated Financial Statements.\n',
 'The Company designs, manufactures and markets mobile communication and media devices and personal computers, and sells a variety of related software, services, accessories, networking solutions and third-party digital content and applications. The Company’s products and services include iPhone, iPad, Mac, Apple Watch, Apple TV, a portfolio of consumer and professional software applications, iOS, macOS, watchOS and tvOS operating systems, iCloud, Apple Pay and a variety of accessory, service and support offerings. The Company sells and delivers digital content and applications through the iTunes Store, App Store, Mac App Store, TV App Store, iBooks Store and Apple Music (collectively “Digital Content and Services”). The Company sells its products worldwide through its retail stores, online stores and direct sales force, as well as through third-party cellular network carriers, wholesalers, retailers and resel

In [0]:
def meanshift(doc_text, num_sents):
  mean_shift = MeanShift().fit(elmo_train)
  # for each cluster, it finds the sentences associated with it,
  # and chooses the median one
  avg = []
  for j in range(num_sents):
      idx = np.where(mean_shift.labels_ == j)[0]
      avg.append(np.median(idx))
  closest, _ = pairwise_distances_argmin_min(mean_shift.cluster_centers_, elmo_train)
  ordering = sorted(range(num_sents), key=lambda k: avg[k])
  summary = [doc_text[closest[idx]] for idx in ordering]
  return summary

meanshift(sentences, 10)

['Basic earnings per share is computed by dividing income available to common shareholders by the weighted-average number of shares of common stock outstanding during the period. Diluted earnings per share is computed by dividing income available to common shareholders by the weighted-average number of shares of common stock outstanding during the period increased to include the number of additional shares of common stock that would have been outstanding if the potentially dilutive securities had been issued. Potentially dilutive securities include unvested restricted stock units (“RSUs”), unvested restricted stock, outstanding stock options and shares to be purchased by employees under the Company’s employee stock purchase plan. The dilutive effect of potentially dilutive securities is reflected in diluted earnings per share by application of the treasury stock method. Under the treasury stock method, an increase in the fair market value of the Company’s common stock can result in a g